# Script to calculate the relevance of twitter friends and follower networks

# Install requirements via pip etc.

Chose to run local or in colab

In [ ]:
run_in = 'local'
#run_in = 'colab'

In [ ]:
if run_in == 'local':
  import csv
  import os
  import pandas as pd
  import numpy as np
  import tweepy
  import datetime
  import time
  import json
  import sys
  import traceback
  import string
  import nltk
  from nltk.tokenize import word_tokenize
  from nltk.corpus import stopwords
  from nltk.stem.porter import PorterStemmer
  import pprint
  pp = pprint.PrettyPrinter(indent=4)
  print('Local runtime has packages')
  from fuzzywuzzy import fuzz
  
if run_in == 'colab':
  print('Downloading packages has packages')
  #!pip install pandas==0.23.4
  !pip install tweepy
  !pip install xlrd
  !pip install XlsxWriter
  import csv
  import os, os.path
  import pandas as pd
  import numpy as np
  import tweepy
  import time
  import json
  import sys
  import time
  from datetime import datetime
  import traceback
  import string
  import nltk
  nltk.download("popular")
  from nltk.tokenize import word_tokenize
  from nltk.corpus import stopwords
  from nltk.stem.porter import PorterStemmer
  import pprint
  pp = pprint.PrettyPrinter(indent=4)
  !pip install fuzzywuzzy
  from fuzzywuzzy import fuzz

In [ ]:
pd.__version__

## Settings

In [ ]:
#Define what to analyse and whether to clean:
#analyse = "bi_directional"
analyse = "whole network"

version_id = "V14"

clean_existing = True # when true, the startups for which a file already exists will be dropped

## Define your personal path to the folder we will work in

In [ ]:
if run_in == 'local':
  base_path = "/users/USERNAME/Google Drive/USERNAME/"
if run_in == 'colab':
  base_path = "/content/drive/My Drive/USERNAME/"
  from google.colab import drive
  drive.mount('/content/drive')
  
print('base_path set to:')
print(base_path)

os.listdir(base_path)

## Import all company names

In [ ]:
# already_analysed_companies=[]
# if clean_existing:
#     if analyse == 'bi_directional':
#       network_relevance_output_path = base_path + "Analysis/Network Relevance/Companies bi_directional/"
#       already_analysed_companies = [] # to be dropped

#       for root, dirs, files in os.walk(network_relevance_output_path):
#         for file in files:
#             if file.endswith(".json"):
#                 #print(os.path.splitext(file)[0])
#                 already_analysed_companies.append(os.path.splitext(file)[0].replace("_bi_directional", ""))
    
#     if analyse == 'whole network':
#       network_relevance_output_path = base_path + "Analysis/Network Relevance/Companies Whole Network/"
#       for root, dirs, files in os.walk(network_relevance_output_path):
#         for file in files:
#             if file.endswith(".json"):
#                 #print(os.path.splitext(file)[0])
#                 already_analysed_companies.append(os.path.splitext(file)[0])
# print(already_analysed_companies)
# print(len(already_analysed_companies))

In [ ]:
all_startup_file = os.path.join("/users/USERNAME/Google Drive/USERNAME/Pitchbook_Crunchbase_Tracxn_Raw/Don't touch V14/V14.xlsx")
#all_startup_file = os.path.join(base_path + "Pitchbook_Crunchbase_Tracxn_Raw/Don't touch V14/V14.xlsx")
print("all_startup_file_path is: " + all_startup_file + " and the file exists: " + str(os.path.isfile(all_startup_file)))
company_name_df = pd.read_excel(all_startup_file, 'V14df_all', usecols = "A", skiprows=0).drop_duplicates()
company_name_df.reset_index(drop=True, inplace=True)
len_original = len(company_name_df)
#company_name_df.head()

print("We have found", str(len_original), "startups")

already_analysed_companies=[]
if clean_existing:
    if analyse == 'bi_directional':
      network_relevance_output_path = base_path + "Analysis/Network Relevance/Companies bi_directional/"
      already_analysed_companies = [] # to be dropped

      for root, dirs, files in os.walk(network_relevance_output_path):
        for file in files:
            if file.endswith(".json"):
                #print(os.path.splitext(file)[0])
                already_analysed_companies.append(os.path.splitext(file)[0].replace("_bi_directional", ""))
    
    if analyse == 'whole network':
      network_relevance_output_path = base_path + "Analysis/Network Relevance/Companies Whole Network/"
      for root, dirs, files in os.walk(network_relevance_output_path):
        for file in files:
            if file.endswith(".json"):
                #print(os.path.splitext(file)[0])
                already_analysed_companies.append(os.path.splitext(file)[0])
                
    print(already_analysed_companies)
    print(len(already_analysed_companies))
    
    df_already_analysed = pd.DataFrame()
    df_already_analysed["company_name"] = already_analysed_companies
    df_already_analysed["analysed"] = "analysed"

    company_name_df = company_name_df.merge(df_already_analysed, on='company_name', how='left')
    company_name_df.fillna("not analysed", inplace = True)
    company_name_df = company_name_df.loc[company_name_df['analysed'] != "analysed"]
    len_cleaned = len(company_name_df)
    print("After cleaning already analysed companies (", str(len_original-len_cleaned), ") we are left with", str(len_cleaned), "companies")

company_name_df.head()

In [ ]:
#company_name_df = company_name_df.loc[company_name_df['company_name'] == "9fin"]
#company_name_df

In [ ]:
#company_name_df = company_name_df[:100]
#company_name_df

## start_up database path
This is the folder with .txt file for each start-up that contains all information:

In [ ]:
start_up_db_path = os.path.join(base_path + "Database/")
# os.listdir(start_up_db_path)

## user_ids database path
This is the folder with .txt file for each user that contains all details and tweets:

In [ ]:
user_details_path = "/Users/USERNAME/Desktop/Mining local/USERNAME/outputs/user_details/"
# os.listdir(user_details_path)

In [ ]:
user_tweets_path = "/Users/USERNAME/Desktop/Mining local/USERNAME/outputs/user_tweets/"
# os.listdir(user_tweets_path)

## Def text_cleaner(text)

In [ ]:
def text_cleaner(text):
   # 1 Take all types of hyphens apart from U+002D 
   de_hyphened_em = str.replace(text, "—", " ")
   de_hyphened_en = str.replace(de_hyphened_em, "–", " ")
   # 2 Tokenize
   tokens = word_tokenize(de_hyphened_en)
   tokens = [item.replace("—", " ") for item in tokens]
   # 3 Remove puctuation
   table = str.maketrans('', '', string.punctuation)
   stripped = [w.translate(table) for w in tokens]
   # Remove lonely letters
   no_single = [w for w in stripped if len(w) > 1]
   # 4 Lower Case
   low_words = [word.lower() for word in no_single]
   # 5 Not Alpha
   no_alpha = [w for w in low_words if w.isalpha()]
   # 6 Delete Stop Words
   stop_words = set(stopwords.words('english'))
   no_stop = [w for w in no_alpha if not w in stop_words]
   # 10 Stem Words
   porter = PorterStemmer()
   stemmed = [porter.stem(word) for word in no_stop] 
   
   return stemmed

## Def get_data(start_up)

In [ ]:
def get_data_company(start_up, clean = True):
  """Function which takes in a startup name in the form of a string ("roborace") returns the descriptions and website content of the startup"""
  with open(start_up_db_path + start_up + '.json', 'r') as startup_json:
        try:
          data = json.load(startup_json)
          company_name = data[0]
          company_dict = data[1]
          website_content = company_dict["website_content"].replace("{", "").replace("}", "").replace("'", "").replace(",", "").split()
        except AttributeError: 
          print("There was no website for this company")
          website_content = []
        
        combined_desc = []
        
        try:
          combined_desc.extend(data[1]["desc_pitchbook"].lower().split())
          #print(1)
          #print(combined_desc)
        except: KeyError
    
        try:
          combined_desc.extend(data[1]["desc_crunchbase"].lower().split())
          #print(2)
          #print(combined_desc)
        except: KeyError
        
        try:
          combined_desc.extend(data[1]["desc_tracxn"].lower().split())
          #print(3)
        except: KeyError
            
        company_match_content = website_content + combined_desc
        company_match_content = set(company_match_content)
  #print(company_name, company_dict, company_match_content)
        if clean:
            cmc = []
            for word in company_match_content:
              cmc += text_cleaner(word)
        else:
            cmc = company_match_content

  return(company_name, company_dict, cmc)
  

## Def get_data(user)

In [ ]:
def get_data_user(user_id, clean=True):
  """Function which takes in a user in the form of a string id ("123382") returns the tweets and description of the user"""
  u = user_id
  #print(u)
  su = str(u)[:2] # takes the first two digits of the user id to find the folder
  user_detail = user_details_path + su + "/" + str(u) + ".json"
  user_tweets = user_tweets_path + su + "/" + str(u) + ".txt" # neu hinzugefügt

  #print(user_detail)
  #print(user_tweets)

  if os.path.isfile(user_detail)==True:
    #print("file does exist")
    with open(user_detail) as u_json:
        u_data = json.load(u_json)
        if u_data == 'error':
          #print("user has no description or was not found") 
          u_desc = []
        else:  
          #print("Step 1 u_data: ",u_data)
          u_desc = u_data["description"].replace("#", "").split() # replace "#" so that they do not get cleaned by the text_cleaner
          #print(type(u_desc))
          #print("Step 2: ", u_desc)
          #print(type(u_desc))
  else:
    u_desc = []
    #print("file does not exist")

  #user_tweets = "/content/drive/My Drive/Mark/outputs/user_tweets/84/844909155857129473.txt"
  if os.path.isfile(user_tweets)==True:
    #print("file does exist")
    with open(user_tweets) as ut_txt:
        ut_data = ut_txt.readlines()
        ut_data = ut_data
        u_tweet_content = []
        #print("ut_data:", ut_data)
        for line in ut_data:
          line = line[2:-1]
          line = line.replace('RT ', '')
          line = line.replace('\\n', '')
          line = line.replace('#', '') # replace "#" so that they do not get cleaned by the text_cleaner
          line = line.replace('\\', ' ')
          line = line.replace('http', '')
          u_tweet_content.append(line)
          #print(type(u_tweet_content))
    #u_tweet_content = ' '.join(u_tweet_content)
    #print("tweet_content of", u, u_tweet_content)
    #print(type(u_tweet_content))
  else:
    u_tweet_content = []
    #print("file does not exist")
    
  user_match_content =  u_desc + u_tweet_content 
  #print("user_match_content:", user_match_content)
  
  if clean:
    umc = []
    for word in user_match_content:
      umc += text_cleaner(word)
  else:
    umc = user_match_contet
  return(umc)

For every startup we want to count the most matched words and put them in a histogram

## Def wordmatcher (lst1=[], lst2=[]

In [ ]:
def word_comp(w1, w2, tol=85):
  """compares two words and returns true if they are the same"""  
  if int(fuzz.ratio(w1, w2)) > tol:
    #print(w1, w2)
    return True
  else:
    return False

def wordmatcher(lst1=[], lst2=[]):
  """compares two lists and returns the matches"""  
  #lst1 = ["cheese", "apples", "goat"]
  #lst2 = ["cheese", "cheese", "goat", "boat"]
  matches ={}
  for w1 in lst1:
    for w2 in lst2:
      if word_comp(w1, w2):
        if w1 not in matches:
          matches[w1] = 0
        matches[w1] += 1
  return matches

def wordmatcher_set(lst1=[], lst2=[]):
  """compares two lists and returns the matches"""  
  unions = list(set(lst1).intersection(set(lst2)))
  #lst1 = ["cheese", "apples", "goat"]
  #lst2 = ["cheese", "cheese", "goat", "boat"]
  matches ={}
  for word in unions:
    matches[word] = lst1.count(word)
  return matches

# Match all startups with all their followers and friends

## analyse == "whole network":

In [ ]:
print("This part of the script STARTED at:", datetime.datetime.now())

In [ ]:
if analyse == "whole network":

  versions = ["V"+str(i) for i in range(1,15)]
  # open all paths to all startup.txt files which contain dictionaries  
  overall_dict = {}
  c_counter = 0
  for index,row in company_name_df.iterrows():
      c_counter += 1
      start_up = row['company_name']
      company_name, company_dict, company_match_content = get_data_company(start_up)
      print("start_up: " + start_up)
      #print("company_dict: " + str(company_dict))
      #print("company_dict: " + str(company_dict.keys()))
      #print("company_match_content: " + str(company_match_content))
      #print("company_match_content: " + str(company_match_content))
      #overall_dict = {}
      overall_dict[company_name] = {}
      print("Company", start_up,":", str(c_counter), "/", str(len(company_name_df)), str(datetime.datetime.now()))

      # for a given startup iterate over the versions and obtain the friends_ids  
      for version in versions:
        print(version,"/",versions[-1])
        overall_dict[company_name][version] = {}

        if version in company_dict:
          for ffid in ["friends", "followers"]:
            print(ffid)
            if ffid + "_ids" in company_dict[version]:
                overall_dict[company_name][version][ffid] = {}
                user_ids = company_dict[version][ffid+"_ids"]
                user_ids = [str(u) for u in user_ids]
                #print("user_ids:",user_ids)

                counter = 0
                for u in user_ids:
                  counter += 1
                  if not counter % 50:
                    print("For", company_name, "User:", str(counter) + "/" + str(len(user_ids)))
                  #print("u:", u)
                  #print("Now we analyse the individual users")
                  user_match_content = get_data_user(u)
                  #print(company_name, version, ffid, u)
                  #print("user_match_content is:", user_match_content)
                  #print("clean text:", text_cleaner(user_match_content))
                  # match words between stemmed desc+tweets AND startupdesc+website (match_content)
                  #matched = wordmatcher_set(user_match_content, company_match_content)
                  matched = wordmatcher_set(user_match_content, company_match_content)
                  #print("matched:",matched)
                    
                  sum_all_matches = 0
                  num_matches = int(np.sum(list(matched.values()))) #number of matches per user
                  sum_all_matches += num_matches #number of matches across users per company

                  # populates the dictrionary per follower
                  overall_dict[company_name][version][ffid][u] = {}
                  overall_dict[company_name][version][ffid][u]["word matches"] = matched
                  overall_dict[company_name][version][ffid][u]["sum word matches"] = num_matches

                word_stats = {} # Used to save absolute number of matches per occuring word
                #total_matched_words # Total number of matched words across all users
                user_matches = {} # Total number of matched words per user
                sum_all_users = 0 # the number of users per company that had matches
                word_user_match = {} # Which users had descriptions or tweets that matched certain words
                #print("Initialised: word_stats, user_matches, sum_all_users, word_user_match dictionaries")

                #loop over users constructing word_stats
                for u in user_ids:
                  m = overall_dict[company_name][version][ffid][u]["word matches"]
                  for w in m:
                    if w not in word_stats:
                      word_stats[w] = 0
                    word_stats[w] += m[w]
                    if w not in word_user_match:
                      word_user_match[w] = []
                    word_user_match[w].append(u)
                    user_matches[u] = overall_dict[company_name][version][ffid][u]["sum word matches"]
                total_matched_words = 0
                for w, n in word_stats.items():
                   total_matched_words += n
                for u,n in user_matches.items():
                  if n > 0:
                    sum_all_users += 1

                overall_dict[company_name][version]["startup" + ffid + "stats"] = {}
                overall_dict[company_name][version]["startup" + ffid + "stats"]["word_stats"] = word_stats
                overall_dict[company_name][version]["startup" + ffid + "stats"]["total_matched_words"] = total_matched_words
                overall_dict[company_name][version]["startup" + ffid + "stats"]["user_matches"] = user_matches
                overall_dict[company_name][version]["startup" + ffid + "stats"]["sum_all_users"] = sum_all_users
                overall_dict[company_name][version]["startup" + ffid + "stats"]["word_user_matches"] = word_user_match

      #pp.pprint(overall_dict) 
      for key, value in overall_dict.items():
        with open(base_path + 'Analysis/Network Relevance/Companies Whole Network/' + start_up + ".json", "w") as su_net_rel_file:
          json.dump((key, value), su_net_rel_file)

In [ ]:
print("This part of the script STOPPED at:", datetime.datetime.now())

# Match all startups with bi-directional followers and friends

## analyse == "bi_directional":

In [ ]:
print("This part of the script STARTED at:", datetime.datetime.now())

In [ ]:
if analyse == "bi_directional":
  versions = ["V"+str(i) for i in range(1,15)]
  #versions = ["V14"]
  #print("versions:",versions)
  # open all paths to all startup.txt files which contain dictionaries  
  c_counter = 0
  overall_dict = {}
  for index,row in company_name_df.iterrows():
      c_counter += 1
      start_up = row['company_name']
      company_name, company_dict, company_match_content = get_data_company(start_up)
      print("start_up: " + start_up)
      #print("company_dict: " + str(company_dict))
      #print("company_dict: " + str(company_dict.keys()))
      #print("got company_match_content")
      #print("company_match_content: " + str(company_match_content))
      #overall_dict = {}
      overall_dict[company_name] = {}
      print("Company", start_up,":", str(c_counter), "/", str(len(company_name_df)), str(datetime.datetime.now()))

      # for a given startup iterate over the versions and obtain the friends_ids  
      for version in versions:
        print(version,"/",versions[-1])
        overall_dict[company_name][version] = {}

        if version in company_dict:
          bi_direct_nodes_version = []
          with open("/users/USERNAME/Google Drive/USERNAME/Pitchbook_Crunchbase_Tracxn_Raw/Don't touch " + version + "/" + version + "_bi_direct_nodes.txt") as node_file:
            #print("Found users in company dict")
            for line in node_file:
              #print("line:", line)
              bi_direct_nodes_version.append(line.strip())
             
          #######here i have to check which ones are bidirectional set(friends)intersection(set(followers))   
          try:
              friend_ids = company_dict[version]["friends_ids"]
              follower_ids = company_dict[version]["followers_ids"]
              fnf_ids = user_ids2 = list(set(friend_ids).intersection(set(follower_ids)))
              user_ids = fnf_ids
                    
              overall_dict[company_name][version]["fnf"] = {}
              counter = 0
              for u in user_ids:
                counter += 1
                if not counter % 50:
                  print("For", company_name, "User:", str(counter) + "/" + str(len(user_ids)))
                #print("u:", u)
                #print("Now we analyse the individual users")
                user_match_content = get_data_user(u)
                #print(company_name, version, "fnf", u)
                #print("user_match_content is:", user_match_content)
                #print("clean text:", text_cleaner(user_match_content))
                # match words between stemmed desc+tweets AND startupdesc+website (match_content)
                matched = wordmatcher_set(user_match_content, company_match_content)
                #matched = wordmatcher(user_match_content, company_match_content)
                #print("matched:",matched)  

                sum_all_matches = 0
                num_matches = int(np.sum(list(matched.values()))) #number of matches per user
                sum_all_matches += num_matches #number of matches across users per company

                # populates the dictrionary per follower
                overall_dict[company_name][version]["fnf"][u] = {}
                overall_dict[company_name][version]["fnf"][u]["word matches"] = matched
                overall_dict[company_name][version]["fnf"][u]["sum word matches"] = num_matches

              word_stats = {} # Used to save absolute number of matches per occuring word
              #total_matched_words # Total number of matched words across all users
              user_matches = {} # Total number of matched words per user
              sum_all_users = 0 # the number of users per company that had matches
              word_user_match = {} # Which users had descriptions or tweets that matched certain words
              #print("Initialised: word_stats, user_matches, sum_all_users, word_user_match dictionaries")

              #loop over users constructing word_stats
              for u in user_ids:
                m = overall_dict[company_name][version]["fnf"][u]["word matches"]
                for w in m:
                  if w not in word_stats:
                    word_stats[w] = 0
                  word_stats[w] += m[w]
                  if w not in word_user_match:
                    word_user_match[w] = []
                  word_user_match[w].append(u)
                  user_matches[u] = overall_dict[company_name][version]["fnf"][u]["sum word matches"]
              total_matched_words = 0
              for w, n in word_stats.items():
                total_matched_words += n
              for u,n in user_matches.items():
                if n > 0:
                  sum_all_users += 1
              #print("calculated word_stats, user_matches, sum_all_users, word_user_match ready to populate dictionaries")

              overall_dict[company_name][version]["startup" + "fnf" + "stats"] = {}
              overall_dict[company_name][version]["startup" + "fnf" + "stats"]["word_stats"] = word_stats
              overall_dict[company_name][version]["startup" + "fnf" + "stats"]["total_matched_words"] = total_matched_words
              overall_dict[company_name][version]["startup" + "fnf" + "stats"]["user_matches"] = user_matches
              overall_dict[company_name][version]["startup" + "fnf" + "stats"]["sum_all_users"] = sum_all_users
              overall_dict[company_name][version]["startup" + "fnf" + "stats"]["word_user_matches"] = word_user_match
              #print("populated word_stats, user_matches, sum_all_users, word_user_match dictionaries")
          except KeyError:
              print("Either friend_ids or follower_ids couldn't be retrieved")
              
      #pp.pprint(overall_dict) 
      for key, value in overall_dict.items():
        with open(base_path + "Analysis/Network Relevance/Companies bi_directional/" + start_up + "_bi_directional.json", "w") as su_net_rel_file:
          json.dump((key, value), su_net_rel_file)